## Evaluating multi-group promps

In [3]:
import pandas as pd
from labelrepo.projects.participant_demographics import get_participant_demographics

subgroups = get_participant_demographics(include_locations=True)

In [4]:
from labelrepo.database import get_database_connection

import pandas as pd

docs_info = pd.read_sql(
    "select pmcid, publication_year, title, text from document",
    get_database_connection(),
)